#**Llama 2**

The Llama 2 is a collection of pretrained and fine-tuned generative text models, ranging from 7 billion to 70 billion parameters, designed for dialogue use cases.

 It outperforms open-source chat models on most benchmarks and is on par with popular closed-source models in human evaluations for helpfulness and safety.

[Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.

#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

#**1: Install All the Required Packages**

In [1]:
!nvidia-smi

Thu Nov 21 17:27:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 1. Install the required packages

In [6]:
# GPU llama-cpp-python

!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1
!pip install llama-cpp-python==0.1.78
!pip install huggingface_hub
!pip install numpy==1.23.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=296593 sha256=4746c4acf6832dca0e84849d189fc03feddfb32d20b8c52caa689e45db273409
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 102.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

# 2. Initialize the model

In [2]:
model_name = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q6_K.bin"

# 3. Import all required libraires

In [3]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# 4. Download the model

In [4]:
model_path = hf_hub_download(repo_id=model_name, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q6_K.bin'

# 5. Load the Model

In [6]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,#cpu_cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [7]:
# to see the number of layes in GPU
lcpp_llm.params.n_gpu_layers

32

# 6.Create Prompt Template

In [8]:
prompt = "Write a Poem about India Like how shakespeare wrote it"
prompt_template = f""" System: You are a helpfull assistant. Answer in a relavent way

User: {prompt}
Assistant :

"""

# 7. Generating the Response

In [9]:
response = lcpp_llm(prompt = prompt_template,
                    max_tokens=256,
                    temperature=0.5,
                    top_p=0.95,
                    repeat_penalty=1.2,
                    top_k=50,
                    echo=True)

In [10]:
print(response)

{'id': 'cmpl-910a2599-778b-4e62-becc-20f10449f724', 'object': 'text_completion', 'created': 1732288976, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q6_K.bin', 'choices': [{'text': " System: You are a helpfull assistant. Answer in a relavent way\n\nUser: Write a Poem about India Like how shakespeare wrote it\nAssistant :  \n\nOh, fairest India, land of spices and gold,\nWhere ancient temples rise to touch the sky so bold,\nA land where culture meets the modern age,\nWhere technology doth flourish in every stage.\n\nThy people, oh so diverse and bright,\nWith customs, languages, and beliefs so tight,\nFrom the snow-capped mountains to the sea so blue,\nThis land of contrasts, a tale anew.\n\nIn thy bustling cities, a world so grand,\nSkyscrapers pierce the sky, a modern land,\nYet in the villages, tradition doth reign,\nA glimpse into history's ancient refrain.\n\nThe Taj Mahal, 

In [11]:
print(response["choices"][0]["text"])

 System: You are a helpfull assistant. Answer in a relavent way

User: Write a Poem about India Like how shakespeare wrote it
Assistant :  

Oh, fairest India, land of spices and gold,
Where ancient temples rise to touch the sky so bold,
A land where culture meets the modern age,
Where technology doth flourish in every stage.

Thy people, oh so diverse and bright,
With customs, languages, and beliefs so tight,
From the snow-capped mountains to the sea so blue,
This land of contrasts, a tale anew.

In thy bustling cities, a world so grand,
Skyscrapers pierce the sky, a modern land,
Yet in the villages, tradition doth reign,
A glimpse into history's ancient refrain.

The Taj Mahal, a wonder to behold,
A symbol of love that time doth unfold,
The Ganges River, where souls are cleansed,
A sacred place, where the divine is disclosed.

Oh India, land of mystery and might,
Thy secrets hidden deep within thy sight,
A land so vast, a story to be told,
A tale of diversity, a treasure to behold.


In [18]:
prompt = "Write a python program to develope a Machine Learning model using Random forest General code or lets take an iris dataset."
prompt_template = f""" System: You are a helpfull assistant. Answer in a relavent way

User: {prompt}
Assistant :

"""

In [19]:
response = lcpp_llm(prompt = prompt_template,
                    max_tokens=256,
                    temperature=0.5,
                    top_p=0.95,
                    repeat_penalty=1.2,
                    top_k=50,
                    echo=True)

Llama.generate: prefix-match hit


In [20]:
print(response["choices"][0]["text"])

 System: You are a helpfull assistant. Answer in a relavent way

User: Write a python program to develope a Machine Learning model using Random forest General code or lets take an iris dataset.
Assistant :  

Sure, I can assist you with that! To develop a machine learning model using random forests, we will use the scikit-learn library in Python. Here is some general code to get started:
```
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Load the iris dataset
iris = pd.read_csv('iris.csv')

# Preprocess the data
X = iris.drop(['class'], axis=1)  # features
y = iris['class']  # target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest classifier on the training set
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model on the testi